In [35]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [56]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
y_test, tX_test, ids = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

In [31]:
i_PRI = 22

y_0 =  y[tX[:, i_PRI]==0]
tx_0 = tX[tX[:, i_PRI]==0]

In [32]:
tx_0_filtered = np.delete(tx_0, [4,5,6,12,22,23,24,25,26,27,28,29], axis=1)

In [34]:
tx_0_filtered

array([[ 1.43905e+02,  8.14170e+01,  8.09430e+01, ...,  3.10820e+01,
         6.00000e-02,  8.60620e+01],
       [ 1.75864e+02,  1.69150e+01,  1.34805e+02, ...,  2.72300e+00,
        -8.71000e-01,  5.31310e+01],
       [ 1.05594e+02,  5.05590e+01,  1.00989e+02, ...,  3.77910e+01,
         2.40000e-02,  1.29804e+02],
       ...,
       [-9.99000e+02,  5.81790e+01,  6.80830e+01, ...,  4.67370e+01,
        -8.67000e-01,  8.04080e+01],
       [ 9.49510e+01,  1.93620e+01,  6.88120e+01, ...,  1.21500e+01,
         8.11000e-01,  1.12718e+02],
       [-9.99000e+02,  7.27560e+01,  7.08310e+01, ...,  4.07290e+01,
        -1.59600e+00,  9.94050e+01]])

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)